In [15]:
from itertools import count
import random
import os
import re
import pandas as pd

string = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789~!@#$%^&*()_+`-=[]\\;',./{}|:\"<>?"
letter = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
capitals = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
digit = "0123456789"
special = "~!@#$%^&*()_+`-=[]\\;',./{}|:\"<>?"
types = ["SAVINGS", "CURRENT"]

# genernate input file path
def generate_file_path():
    # vaild or invalid file path
    valid = random.random() > 0.5
    if valid:
        files = os.listdir("FuzzCSV")
        file_path = "FuzzCSV/" + files[random.randint(0,len(files)-1)]
    else:
        rand_str = ""
        for i in range(10,100):
            rand_str += str[random.randint(0,61)]
        file_path = rand_str
    return file_path

def random_generate_csv(file_name, fuzz_rate, col_num_low, col_num_high, not_ordered):

    Allheaders = ["Customer ID#", "Account No.", "Currency", "Type", "Balance"]

    # randomly choose which cols used to generate
    col_num = random.randint(1,4)
    col_idx = random.sample([0,1,2,3,4], col_num)
    sel_headers = [Allheaders[i] for i in col_idx]
    
    fuzzed_data = pd.DataFrame(columns = sel_headers)

    if (len(sel_headers)==0):
        fuzzed_data.to_csv(f"FuzzCSV/{file_name}", index=False)

    # decide how many rows will generate
    row_num = random.randint(col_num_low, col_num_high)
        
    id_index = 1

    for i in range(row_num):
        row = {}
        for header in sel_headers:
            if header == "Customer ID#":
                # each entry has probability to be empty
                empty = random.random() < fuzz_rate
                if empty:
                    row["Customer ID#"] = ""
                else:
                    id = "ID" + str(id_index)
                    row["Customer ID#"] = id
                id_index += 1


            elif header == "Account No.":
                empty = random.random() < fuzz_rate
                if empty:
                    row["Account No."] = ""
                else:
                    account = ""
                    # generate 3 captial letters first
                    for i in range(3):
                        account += capitals[random.randint(0,25)]
                    # followd by 8 digits
                    for i in range(8):
                        account += digit[random.randint(0,9)]
                    row["Account No."] = account

            elif header == "Currency":
                empty = random.random() < fuzz_rate
                if empty:
                    row["Currency"] = ""
                else:
                    currency = ""
                    # generate 3 captial letters
                    for i in range(3):
                        currency += capitals[random.randint(0,25)]
                    row["Currency"] = currency

            elif header == "Type":
                empty = random.random() < fuzz_rate
                if empty:
                    row["Type"] = ""
                else:
                    type_choice = random.random()<=0.5
                    if type_choice:
                        tp = "SAVINGS"
                    else:
                        tp = "CURRENT"
                    row["Type"] = tp
            
            elif header == "Balance":
                empty = random.random() < fuzz_rate
                if empty:
                    row["Balance"] = ""
                else:
                    balance = ""
                    # length of balance
                    le = random.randint(0,8)
                    for i in range(le):
                        # randomly choose a digit
                        if i==0:
                            balance += digit[random.randint(1,9)]
                        else:
                            balance += digit[random.randint(0,9)]
                    row["Balance"] = balance

        fuzzed_data = fuzzed_data.append(row, ignore_index=True)
    
    # id not ordered
    if not_ordered:
        fuzzed_data = fuzzed_data.sample(frac = 1)
    
    fuzzed_data.to_csv(f"FuzzCSV/{file_name}", index=False)
    
# generate csv files
for i in range(30):
    fuzz_rate = random.random() / 10
    generated_file_name = "csvfile_fuzzed " + str(i+1) + ".csv"
    if random.random() < 0.5:
        not_ordered = True
    else:
        not_ordered = False
    random_generate_csv(generated_file_name, fuzz_rate, 1, 100, not_ordered)